In [9]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from datetime import datetime
import os
import glob
from IPython.display import display, HTML

class HandTracker:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_draw = mp.solutions.drawing_utils
        self.results_data = []
        self.frame_summary = []

    def process_video(self, video_path, output_csv_path):
        """Process a single video and save only CSV output (no video output)"""
        print(f"Processing video: {video_path}")
        
        if not os.path.exists(video_path):
            print(f"Error: Video file not found at {video_path}")
            return False
        
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
        
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                return False
            
            fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
            print(f"Video properties: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))} @ {fps}fps")
            print(f"Total frames: {total_frames}")
            
            frame_count = 0
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                hand_results = self.hands.process(frame_rgb)
                hand_data = self.extract_hand_data(hand_results, frame_count, fps)

                left_hand_present = any(item['hand_label'] == 'Left' for item in hand_data)
                right_hand_present = any(item['hand_label'] == 'Right' for item in hand_data)
                
                # Ensure at most one hand is marked as holding the item
                if left_hand_present and right_hand_present:
                    left_hand_present = False  # Give priority to the right hand
                
                self.frame_summary.append({
                    'frame': frame_count,
                    'timestamp': frame_count / fps,
                    'left_hand_present': left_hand_present,
                    'right_hand_present': right_hand_present,
                })

                self.results_data.extend(hand_data)
                frame_count += 1

                if frame_count % 30 == 0:
                    print(f"Processed {frame_count} frames ({int(frame_count/total_frames*100)}%)")

            cap.release()
            
            if frame_count > 0:
                # Save CSV output
                success = self.save_to_csv(output_csv_path)
                if success:
                    print(f"CSV file saved: {output_csv_path}")
                    return True
                else:
                    print("Failed to save CSV file")
                    return False
            else:
                print("No frames were processed.")
                return False
                
        except Exception as e:
            print(f"Error processing video: {str(e)}")
            return False

    def extract_hand_data(self, results, frame_number, fps):
        hand_data = []
        if results.multi_hand_landmarks:
            for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                hand_label = results.multi_handedness[hand_idx].classification[0].label
                for landmark_idx, landmark in enumerate(hand_landmarks.landmark):
                    hand_data.append({
                        'frame': frame_number,
                        'timestamp': frame_number / fps,
                        'hand_index': hand_idx,
                        'hand_label': hand_label,
                        'landmark_index': landmark_idx,
                        'landmark_name': self.get_landmark_name(landmark_idx),
                        'x': landmark.x,
                        'y': landmark.y,
                        'z': landmark.z,
                    })
        return hand_data
    
    def get_landmark_name(self, idx):
        """Get the name of the hand landmark based on its index."""
        landmark_names = [
            "WRIST",
            "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",
            "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",
            "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",
            "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",
            "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
        ]
        
        if 0 <= idx < len(landmark_names):
            return landmark_names[idx]
        return f"UNKNOWN_{idx}"

    def save_to_csv(self, output_path):
        try:
            # Create a merged DataFrame
            landmarks_df = pd.DataFrame(self.results_data)
            summary_df = pd.DataFrame(self.frame_summary)
            
            print(f"Landmarks data: {len(landmarks_df)} rows")
            print(f"Summary data: {len(summary_df)} rows")
            
            # Merge the summary data with landmark data
            if not landmarks_df.empty and not summary_df.empty:
                # Join the frames that have hand landmarks with the summary info
                merged_df = pd.merge(
                    landmarks_df, 
                    summary_df[['frame', 'left_hand_present', 'right_hand_present']], 
                    on='frame', 
                    how='left'
                )
            elif not landmarks_df.empty:
                merged_df = landmarks_df
                merged_df['left_hand_present'] = merged_df['hand_label'] == 'Left'
                merged_df['right_hand_present'] = merged_df['hand_label'] == 'Right'
            else:
                # If no hands were detected in the video
                merged_df = summary_df
                
            # Save to CSV
            merged_df.to_csv(output_path, index=False)
            return True
            
        except Exception as e:
            print(f"Error saving CSV: {str(e)}")
            return False

def find_all_videos():
    """Find all video files in the current directory"""
    current_dir = os.getcwd()
    print(f"Looking for videos in: {current_dir}")
    
    # Search for common video extensions in the current directory
    video_extensions = ['*.mp4', '*.avi', '*.mov', '*.mkv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(glob.glob(os.path.join(current_dir, ext)))
    
    if video_files:
        print(f"Found video files: {video_files}")
        return video_files
    else:
        print("No video files found in the current directory.")
        return []

def explain_coordinates():
    """Explain the meaning of x, y, z coordinates in the MediaPipe hand tracking."""
    explanation = """
    <h3>Coordinate System in MediaPipe Hand Tracking</h3>
    <p>The x, y, z coordinates from the hand tracking represent normalized 3D coordinates:</p>
    <ul>
        <li><strong>x</strong>: Horizontal position in the image (0.0 is the left edge, 1.0 is the right edge)</li>
        <li><strong>y</strong>: Vertical position in the image (0.0 is the top edge, 1.0 is the bottom edge)</li>
        <li><strong>z</strong>: Depth from the camera (relative to the wrist landmark). Negative values are closer to the camera.</li>
    </ul>
    <p>These coordinates are normalized to the image dimensions, making them independent of the input image resolution.</p>
    """
    return HTML(explanation)

def process_all_videos():
    """Process all videos in the current directory and save CSV outputs in an 'output' folder"""
    print("Looking for video files...")
    video_paths = find_all_videos()
    
    if not video_paths:
        print("No videos to process.")
        return None
    
    # Create output directory
    current_dir = os.getcwd()
    output_dir = os.path.join(current_dir, "output")
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output directory created: {output_dir}")
    
    results = []
    for video_path in video_paths:
        # Reset the tracker for each video
        tracker = HandTracker()
        
        # Setup file paths for output
        video_filename = os.path.basename(video_path)
        video_name = os.path.splitext(video_filename)[0]
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_csv_path = os.path.join(output_dir, f"{video_name}_data_{timestamp}.csv")
        
        # Process the video
        print(f"\nProcessing video: {video_filename}")
        success = tracker.process_video(video_path, output_csv_path)
        
        if success:
            print(f"Processing complete for {video_filename}!")
            print(f"CSV output: {output_csv_path}")
            results.append({
                'video_name': video_name,
                'original_path': video_path,
                'output_csv': output_csv_path
            })
        else:
            print(f"Processing failed for {video_filename}.")
    
    return results

# Main function for Jupyter notebook
def run_hand_tracking():
    """Run the hand tracking process on all videos in the current directory."""
    print("Starting hand tracking process...")
    
    results = process_all_videos()
    
    if results:
        explain = explain_coordinates()
        display(explain)
        
        # Display a preview of the CSV data for each video
        for result in results:
            try:
                data = pd.read_csv(result['output_csv'])
                print(f"\nPreview of the CSV data for {result['video_name']}:")
                display(data.head())
                print(f"Total rows: {len(data)}")
            except Exception as e:
                print(f"Could not display CSV preview for {result['video_name']}: {str(e)}")
    
    return results

# Run the hand tracking process
run_hand_tracking()

Starting hand tracking process...
Looking for video files...
Looking for videos in: /Users/mianwang/Documents/N/DS5500/videos
Found video files: ['/Users/mianwang/Documents/N/DS5500/videos/00013.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00007.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00006.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00012.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00004.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00010.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00011.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00005.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00001.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00015.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00014.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00016.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00002.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00003.mp4', '/Users/mianwang/Documents/N/DS5500/videos/00017.mp4', '/Users/mianwang/Documents/N/

I0000 00:00:1742691311.603528 1331623 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2 Pro
W0000 00:00:1742691311.619009 1426052 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742691311.628849 1426052 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processed 30 frames (7%)
Processed 60 frames (14%)
Processed 90 frames (21%)
Processed 120 frames (28%)
Processed 150 frames (35%)
Processed 180 frames (42%)
Processed 210 frames (49%)
Processed 240 frames (56%)
Processed 270 frames (63%)
Processed 300 frames (70%)
Processed 330 frames (78%)
Processed 360 frames (85%)
Processed 390 frames (92%)
Processed 420 frames (99%)
Landmarks data: 14868 rows
Summary data: 423 rows
CSV file saved: /Users/mianwang/Documents/N/DS5500/videos/output/00013_data_20250322_205511.csv
Processing complete for 00013.mp4!
CSV output: /Users/mianwang/Documents/N/DS5500/videos/output/00013_data_20250322_205511.csv

Processing video: 00007.mp4
Processing video: /Users/mianwang/Documents/N/DS5500/videos/00007.mp4
Video properties: 360x640 @ 30fps
Total frames: 662


I0000 00:00:1742691322.556882 1331623 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83), renderer: Apple M2 Pro
W0000 00:00:1742691322.563696 1426181 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742691322.573869 1426185 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Processed 30 frames (4%)
Processed 60 frames (9%)
Processed 90 frames (13%)
Processed 120 frames (18%)
Processed 150 frames (22%)
Processed 180 frames (27%)
Processed 210 frames (31%)
Processed 240 frames (36%)
Processed 270 frames (40%)
Processed 300 frames (45%)
Processed 330 frames (49%)
Processed 360 frames (54%)
Processed 390 frames (58%)
Processed 420 frames (63%)
Processed 450 frames (67%)
Processed 480 frames (72%)
Processed 510 frames (77%)
Processed 540 frames (81%)
Processed 570 frames (86%)
Processed 600 frames (90%)
Processed 630 frames (95%)
Processed 660 frames (99%)
Landmarks data: 22512 rows
Summary data: 662 rows
CSV file saved: /Users/mianwang/Documents/N/DS5500/videos/output/00007_data_20250322_205522.csv
Processing complete for 00007.mp4!
CSV output: /Users/mianwang/Documents/N/DS5500/videos/output/00007_data_20250322_205522.csv

Processing video: 00006.mp4
Processing video: /Users/mianwang/Documents/N/DS5500/videos/00006.mp4
Video properties: 360x640 @ 30fps
Total

...

,frame,timestamp,hand_index,hand_label,landmark_index,landmark_name,x,y,z,left_hand_present,right_hand_present
0,0,0.0,0,Right,0,WRIST,0.754753,0.638412,-1.099473e-07,False,True
1,0,0.0,0,Right,1,THUMB_CMC,0.717859,0.615728,-1.102773e-02,False,True
2,0,0.0,0,Right,2,THUMB_MCP,0.664687,0.613582,-1.907547e-02,False,True
3,0,0.0,0,Right,3,THUMB_IP,0.621028,0.620886,-2.405320e-02,False,True
4,0,0.0,0,Right,4,THUMB_TIP,0.588883,0.626677,-2.793313e-02,False,True


Total rows: 15078

Preview of the CSV data for 00009:


,frame,timestamp,hand_index,hand_label,landmark_index,landmark_name,x,y,z,left_hand_present,right_hand_present
0,0,0.0,0,Left,0,WRIST,0.235891,0.789658,-2.704542e-08,True,False
1,0,0.0,0,Left,1,THUMB_CMC,0.213119,0.790713,-8.272107e-02,True,False
2,0,0.0,0,Left,2,THUMB_MCP,0.205154,0.811734,-1.173214e-01,True,False
3,0,0.0,0,Left,3,THUMB_IP,0.220483,0.834356,-1.373376e-01,True,False
4,0,0.0,0,Left,4,THUMB_TIP,0.235607,0.860380,-1.541079e-01,True,False


Total rows: 26439


[{'video_name': '00013',
  'original_path': '/Users/mianwang/Documents/N/DS5500/videos/00013.mp4',
  'output_csv': '/Users/mianwang/Documents/N/DS5500/videos/output/00013_data_20250322_205511.csv'},
 {'video_name': '00007',
  'original_path': '/Users/mianwang/Documents/N/DS5500/videos/00007.mp4',
  'output_csv': '/Users/mianwang/Documents/N/DS5500/videos/output/00007_data_20250322_205522.csv'},
 {'video_name': '00006',
  'original_path': '/Users/mianwang/Documents/N/DS5500/videos/00006.mp4',
  'output_csv': '/Users/mianwang/Documents/N/DS5500/videos/output/00006_data_20250322_205538.csv'},
 {'video_name': '00012',
  'original_path': '/Users/mianwang/Documents/N/DS5500/videos/00012.mp4',
  'output_csv': '/Users/mianwang/Documents/N/DS5500/videos/output/00012_data_20250322_205551.csv'},
 {'video_name': '00004',
  'original_path': '/Users/mianwang/Documents/N/DS5500/videos/00004.mp4',
  'output_csv': '/Users/mianwang/Documents/N/DS5500/videos/output/00004_data_20250322_205601.csv'},
 {'vi